In [ ]:
!pip install transformers
!pip install evaluate
!pip install sentencepiece
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, RandomSampler
from transformers import T5Model, T5ForConditionalGeneration, T5TokenizerFast
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Import custom module

In [ ]:
%cd /content/drive/MyDrive/ITMO/T5QA/src
%ls

/content/drive/MyDrive/ITMO/T5QA/src
medqadataset.py  __pycache__/  trainer.py


In [ ]:
from medqadataset import MedQADataset
import trainer
#from trainer import Trainer

In [ ]:
# # For debugging
# import importlib
# importlib.reload(trainer)

In [ ]:
#MODEL_NAME = "cointegrated/rut5-base"    # pre-trained model (first iteration)
MODEL_NAME = "/content/drive/MyDrive/ITMO/T5QA/checkpoints/best_model/MedQAModel_MedQuAD_context_sum_qa_gen_concat/MedQAModel_MedQuAD_context_sum_qa_gen_concat"
TOKENIZER = T5TokenizerFast.from_pretrained(MODEL_NAME, local_files_only=True)
#TOKENIZER = T5TokenizerFast.from_pretrained(MODEL_NAME)
# Set up the T5 model
MODEL = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
# Set up the optimizer
OPTIMIZER = Adam(MODEL.parameters(), lr=0.0001)
Q_LEN = 512    # Question Length
T_LEN = 128    # Target Length
BATCH_SIZE = 4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
DEVICE

'cpu'

### Read Data

In [ ]:
PATH = '/content/drive/MyDrive/ITMO/T5QA'

data = pd.read_csv(PATH + '/data/MedQuAD_context_sum_qa_gen_concat.csv', index_col=0)
data.index = [i for i in range(len(data))]

data

,question,context,answer
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами представляет...
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами встречается ...
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляе...,"мутации в генах jup, dsp, dsc2 и kank2 вызываю..."
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,большинство случаев кератодермии с курчавыми в...
4,что такое синдром кноблоха?,"синдром кноблоха — редкое заболевание, характ...","синдром кноблоха — редкое заболевание, характе..."
...,...,...,...
1831,что такое трансплантация поджелудочной железы?,трансплантация поджелудочной железы — это опер...,операция по пересадке здоровой поджелудочной ж...
1832,что такое сирингомиелия?,"сирингомиелия — редкое заболевание, вызывающее...","редкое заболевание, которое приводит к образов..."
1833,какой тип инсульта встречается реже?,геморрагический инсульт – менее распространенн...,геморрагический
1834,что такое креатинин?,"креатинин — это продукт отходов, вырабатываемы...","продукт жизнедеятельности, вырабатываемый ваши..."


### Example

In [ ]:
sample = data.iloc[0]
encoding = TOKENIZER(
    sample["question"],
    sample["context"],
    max_length=512,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors='pt'
)

answer_encoding = TOKENIZER(
    sample["answer"],
    max_length=32,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)

labels = answer_encoding["input_ids"]

output = MODEL(
    input_ids=encoding["input_ids"],
    attention_mask=encoding["attention_mask"],
    labels=labels
)

In [ ]:
labels = TOKENIZER.decode(labels.flatten(), skip_special_tokens=True)

In [ ]:
sample["answer"]

'кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волосы и во многих случаях повышают риск потенциально опасных для жизни проблем с сердцем. у людей с такими заболеваниями волосы необычно грубые, сухие, тонкие и туго завитые. в некоторых случаях волосы также редкие. шерстистая текстура волос обычно поражает только волосы на коже головы и присутствует с рождения. начиная с раннего возраста, у пораженных людей также развивается ладонно-подошвенная кератодермия — состояние, при котором кожа на ладонях рук и подошвах ног становится толстой, чешуйчатой \u200b\u200bи мозолистой. кардиомиопатия, заболевание сердечной мышцы, представляет собой опасную для жизни проблему со здоровьем, которая может развиться у людей с кератодермией и пушистыми волосами. в отличие от других особенностей этого состояния, признаки и симптомы кардиомиопатии могут не проявляться до подросткового возраста или позже. осложнения кардиомиопатии могут включать наруш

In [ ]:
labels

'кератодермия с пушистыми волосами представляет собой группу связанных состояний, которые поражают кожу и волос'

In [ ]:
output.loss

tensor(1.2580, grad_fn=<NllLossBackward0>)

In [ ]:
sample_q = data.iloc[0]["question"]
sample_a = data.iloc[0]["answer"]

sample_encoding = TOKENIZER(sample_q, sample_a)

In [ ]:
sample_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
preds = [
    TOKENIZER.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    for input_id in sample_encoding["input_ids"]
]

In [ ]:
" ".join(preds)

'что тако е  кера то дерм ия с пуш исты ми волос ами ?   кера то дерм ия с пуш исты ми волос ами представля ет с обой групп у связ анных состояни й ,  которые пора жают кож у  и волос ы  и во мног их случа ях по выш ают риск потенци ально опас ных  для жизни проблем с серд цем . у люд ей с так ими заболевани ями волос ы не обычн о  груб ые , сух ие , тон кие  и ту го зави тые . в не которы х случа ях волос ы так же ред кие . шер сти стая текст ура волос  обычн о пора жает  только волос ы на кож е голов ы  и присут ствует с  рождени я . начина я с ран него возраст а , у по ражен ных люд ей так же развива ется лад онно - по дош вен ная  кера то дерм ия — состояни е , при котор ом кожа на  ладо нях рук  и по дош вах  ног станов ится тол стой ,  чеш уй чат ой  и моз оли стой . кардио ми опати я , заболевани е  сердеч ной мыш цы , представля ет с обой опас ную  для жизни проблем у со здоровь ем , котор ая  может раз виться у люд ей с  кера то дерм ией  и пуш исты ми волос ами . в от личие о

### Dataloaders

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.02, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

print(f"Train shape: {train_data.shape}")
print(f"Validation shape: {val_data.shape}")
print(f"Test shape: {test_data.shape}")

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = MedQADataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(OPTIMIZER, patience=5, verbose=True)

Train shape: (1439, 3)
Validation shape: (360, 3)
Test shape: (37, 3)


### Train and validation loop

In [ ]:
N_EPOCHS = 54
STATE_DICT_PATH = '/content/drive/MyDrive/ITMO/T5QA/checkpoints/best_model/MedQAModel_MedQuAD_context_sum_qa_gen_concat'

trainer_module = trainer.Trainer(
    MODEL,
    N_EPOCHS,
    STATE_DICT_PATH,
    train_loader,
    val_loader,
    OPTIMIZER,
    DEVICE,
    TOKENIZER,
    Q_LEN,
    T_LEN,
    scheduler
)

In [ ]:
trainer_module.fit()

NameError: ignored

### Evaluate the results

In [ ]:
test_sample = test_data.iloc[3]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"
test_answer = f"{test_sample['answer']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question,
   test_answer,
   print_text=True
)

pred

In [ ]:
# Calculate metrics on test
bleu_avg, rouge_avg = trainer_module.calc_avg_metrics(test_data)
print("Test:")
print("BLEU:", bleu_avg)
print("RougeL:", rouge_avg)

Test:
BLEU: 0.8013309065455421
RougeL: 0.8645553032996313


In [ ]:
#loss_filepath = "/content/drive/MyDrive/ITMO/T5QA/data/t5_MedQAD_loss_63.csv"
# Plot loss
trainer_module.plot_loss()

### Inference

In [ ]:
test_sample = test_data.iloc[12]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос:  что вызывает отек лица? также называется: одутловатое лицо; отек лица; лунное лицо; отек лица
Контекст: отек лица это скопление жидкости в тканях лица. отек также может возникнуть на шее и плечах. соображения если отек лица легкий, его может быть трудно обнаружить. сообщите врачу следующее: боль и места, где она болит. как долго сохраняется отек. что делает его лучше или хуже? если у вас есть другие симптомы. причины причины отека лица могут включать в себя: аллергическую реакцию аллергический ринит, сенная лихорадка или укус пчелы. отек квинке реакция на переливание крови целлюлит конъюнктивит воспаление глаз реакция на лекарства, в том числе вызванные аспирином, пенициллином, сульфаниламидами, глюкокортикоидами и др. операции на голове, носу или челюсти травма или травма лица например, ожог недоедание в тяжелых случаях ожирение слюны заболевания желез синусит ячмень с отеком вокруг инфицированного глаза абсцесс зуба уход на дому применяйте холодные компрессы, чтобы уменьшить

In [ ]:
test_sample = test_data.iloc[15]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос:  что такое путаница? также называется: дезориентация; мышление неясное; мысли затуманенное
Контекст: замешательство это неспособность думать так ясно и быстро, как обычно. вы можете чувствовать дезориентацию и испытывать трудности с концентрацией внимания, запоминанием и принятием решений. соображения замешательство может нарастать быстро или медленно с течением времени, в зависимости от причины. во многих случаях спутанность сознания длится недолго и проходит. в других случаях это навсегда и неизлечимо. это может быть связано с бредом или деменцией. спутанность сознания чаще встречается у пожилых людей и часто возникает во время пребывания в больнице. некоторые люди с растерянностью могут вести себя странно или необычно или вести себя агрессивно. причины спутанность сознания может быть вызвана различными проблемами со здоровьем, например :алкогольная или наркотическая интоксикация опухоль головного мозга травма головы или травма головы сотрясение мозгалихорадкадисбаланс жидкос

In [ ]:
test_sample = test_data.iloc[22]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос:  каковы симптомы дивертикулеза? также называется: дивертикула дивертикулез; дивертикулит дивертикулез; дивертикулярная болезнь дивертикулез; желудочнокишечное кровотечение дивертикулез; желудочнокишечное кровотечение дивертикулез; желудочнокишечное кровотечение дивертикулез
Контекст: дивертикулез возникает, когда на внутренней стенке кишечника образуются маленькие выпуклые мешочки или мешочки. эти мешочки называются дивертикулами. чаще всего эти мешочки образуются в толстой кишке толстой кишке. они также могут возникать в тощей кишке и тонкой кишке, хотя это встречается реже. причины дивертикулез реже встречается у людей в возрасте 40 лет и моложе. это чаще встречается у пожилых людей. около половины американцев старше 60 лет страдают этим заболеванием. у большинства людей это случается к 80 годам. никто точно не знает, что вызывает образование этих мешочков. в течение многих лет считалось, что диета с низким содержанием клетчатки может сыграть свою роль. недостаток клетчатки м

In [ ]:
test_sample = test_data.iloc[9]
test_context = f"Контекст: {test_sample['context']}"
test_question = f"Вопрос: {test_sample['question']}"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос:  что вызывает абсцесс?
Контекст: абсцесс это скопление гноя в любой части тела. в большинстве случаев область вокруг абсцесса опухает и воспаляется. причины абсцессы возникают, когда участок ткани заражается, и иммунная система организма пытается бороться с ним и сдерживать его. лейкоциты лейкоциты перемещаются через стенки кровеносных сосудов в область инфекции и собираются в поврежденной ткани. в ходе этого процесса образуется гной. гной это скопление жидкости, живых и мертвых лейкоцитов, мертвых тканей, бактерий или других посторонних веществ. абсцессы могут образовываться практически в любой части тела. кожа, под кожей и зубы являются наиболее распространенными местами. абсцессы могут быть вызваны бактериями, паразитами и инородными веществами. абсцессы на коже легко увидеть. они красные, приподнятые и болезненные. абсцессы в других частях тела могут быть не видны, но они могут вызывать повреждение органов. некоторые типы и локализации абсцессов включают: абдоминальный абсц

In [ ]:
test_context = f"Контекст: Холецистит является воспалительным процессом, который возникает в желчном пузыре. От этой патологии сегодня страдает около 20% жителей всего мира. Причем наиболее подвержены заболеванию женщины старше 50 лет. Воспаление выявляют у людей и другого возраста. Причем для детей и подростков характерна бескаменная форма. Зачастую болезнь диагностируется в развитых странах. Обусловлено это особым образом жизни и пищевым поведением."
test_question = f"Вопрос: Нужно ли обращаться к врачу, если есть признаки холецистита?"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос: Нужно ли обращаться к врачу, если есть признаки холецистита?
Контекст: Холецистит является воспалительным процессом, который возникает в желчном пузыре. От этой патологии сегодня страдает около 20% жителей всего мира. Причем наиболее подвержены заболеванию женщины старше 50 лет. Воспаление выявляют у людей и другого возраста. Причем для детей и подростков характерна бескаменная форма. Зачастую болезнь диагностируется в развитых странах. Обусловлено это особым образом жизни и пищевым поведением.
Ответ: позвоните своему врачу, если есть признаки холецистита


In [ ]:
test_context = f"Контекст: Холецистит является воспалительным процессом, который возникает в желчном пузыре. От этой патологии сегодня страдает около 20% жителей всего мира. Причем наиболее подвержены заболеванию женщины старше 50 лет. Воспаление выявляют у людей и другого возраста. Причем для детей и подростков характерна бескаменная форма. Зачастую болезнь диагностируется в развитых странах. Обусловлено это особым образом жизни и пищевым поведением."
test_question = f"Вопрос: Что такое холецистит?"

pred = trainer_module.predict_answer(
   test_context,
   test_question
)

print(test_question)
print(test_context)
print(pred)

Вопрос: Что такое холецистит?
Контекст: Холецистит является воспалительным процессом, который возникает в желчном пузыре. От этой патологии сегодня страдает около 20% жителей всего мира. Причем наиболее подвержены заболеванию женщины старше 50 лет. Воспаление выявляют у людей и другого возраста. Причем для детей и подростков характерна бескаменная форма. Зачастую болезнь диагностируется в развитых странах. Обусловлено это особым образом жизни и пищевым поведением.
Ответ: воспаление является воспалительным процессом, который возникает в желчном пузыре
